In [ ]:
# Imports
import base64
import json
import os

import requests
import azureml.core
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.model import InferenceConfig
from azureml.core import Environment

from azureml.core.webservice import AciWebservice, Webservice
from azureml.exceptions import WebserviceException


In [ ]:
# The name of the model as it will appear in AzureML
aml_model_name = 'PneumoniaDetection-secure'  # Updating this requires an update to score.py

# The name of the model endpoint to be created in AzureML
aci_service_name = 'pneumonia-detection-onnx' 

# The name of the model as it will appear in AI Builder
aib_model_name = "pneumonia-detection-v1"

# The local path of the parent of the model directory
model_path = '.'  

is_secure = True

In [ ]:
ws = Workspace.from_config()

In [ ]:
# Register an AML MOdel
model_root = os.path.join(model_path, './model')
model = Model.register(workspace=ws,
                       model_path=model_root, 
                       model_name=aml_model_name, 
                       tags={'area': "image", 'type': "classification"}, 
                       )

print(f"Registered model {model.name}, Version {model.version}")

In [ ]:
#Creating an inference configuration

entry_script = os.path.join(model_root, "score.py")
conda_file = os.path.join(model_root, "myenv.yml")

inference_config = InferenceConfig(runtime="python", 
                                   entry_script=entry_script,
                                   conda_file=conda_file)

In [ ]:
service = None

try:
    # get any existing service with the specified name 
    service = Webservice(ws, name=aci_service_name)
except WebserviceException as e:
    print(f"Webservice not found: {aci_service_name}")

# Update the service with the new model if the service exists, otherwise deploy a new service
if service:
    print (f"Updating service {aci_service_name}")
    model = Model(workspace=ws, name=aml_model_name)
    service.update(models=[model], inference_config=inference_config, auth_enabled=is_secure)
else:
    print (f"Deploying new service {aci_service_name}")
    deployment_config = AciWebservice.deploy_configuration(cpu_cores = 2, memory_gb = 2, auth_enabled=is_secure)
    service = Model.deploy(ws, aci_service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

In [ ]:
#validate service using response 
service = Webservice(ws, name=aci_service_name)
uri = service.scoring_uri
image_path = './TestData/test_1.jpeg'
api_key = service.get_keys()[0] # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

with open(image_path, "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode('utf-8')

    request_body = [{'image': encoded_string}]

    request_data = json.dumps({'request': request_body})
    response = requests.post(uri, headers=headers, data=request_data)
    print(response.text)